In [44]:
import pandas as pd
import numpy as np
import os

# read all files
folderpath = r"F:\BOCIF\7688"
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []

for path in filepaths:
    with open(path,errors='ignore', mode='r') as f:
        file = f.readlines()
        all_files.append(file)

In [222]:
# extract date, commission, pledge amount, margin occupied, fund avail, 特殊贡献返还
list_all_date=[]
list_commission =[]
list_date = []
list_特殊贡献返还 = []
list_pledge_amount = []
list_margin_occupied =[]
list_fund_avail = []

for file in all_files:
    date =  file[1].split()[2][5:]
    list_all_date.append(date)
    
    commission = file[14].split()[4]
    list_commission.append(commission)
    
    for line in file:
        if '特殊贡献返还' in line:
            
            date = line.split('|')[1]
            list_date.append(date)
            
            特殊贡献返还 = float(line.split('|')[3])
            list_特殊贡献返还.append(特殊贡献返还)
            
            pledge_amount = float(file[11].split()[-1])
            list_pledge_amount.append(pledge_amount)
            
            margin_occupied = float(file[14].split()[-1])
            list_margin_occupied.append(margin_occupied)
            
            fund_avail = float(file[19].split()[-1])
            list_fund_avail.append(fund_avail)
        
        if '特殊贡献返还-利息' in line:
            print(line)

|20210625|出入金                  |       116023.63|            0.00|特殊贡献返还-利息                         |

|20210928|出入金                  |       449789.11|            0.00|特殊贡献返还-利息                         |

|20211227|出入金                  |       618695.39|            0.00|特殊贡献返还-利息                         |

|20220325|出入金                  |       525575.52|            0.00|特殊贡献返还-利息                         |

|20220627|出入金                  |       551240.29|            0.00|特殊贡献返还-利息                         |

|20220927|出入金                  |       107098.17|            0.00|特殊贡献返还-利息                         |



In [223]:
df_commission = pd.DataFrame(index=list_all_date)
df_commission['手续费'] = list_commission
df_commission = df_commission.truncate(after='20221123') # cannot read files after 20221123
df_commission = df_commission.astype({'手续费': 'float'})
df_commission.index = pd.to_datetime(df_commission.index)
df_commission['year'] = df_commission.index.year
df_commission['month'] = df_commission.index.month
df_commission = df_commission.groupby(['year','month']).sum()
df_commission = df_commission.rename(columns={'手续费':'月手续费'})

In [224]:
list_ratio = []
for i in range(len(list_pledge_amount)):
    if list_pledge_amount[i]>list_margin_occupied[i]:  # compare pledge amount with margin occupied
        list_fund_avail[i] = round(list_fund_avail[i] - (list_pledge_amount[i]-list_margin_occupied[i]),2) # arrive at new fund avail
    list_ratio.append(list_特殊贡献返还[i]/list_fund_avail[i]) # calculate 特殊贡献返还/fund avail

In [226]:
# suppress scientific notation
list_fund_avail_str = []
for i in range(len(list_fund_avail)):
    list_fund_avail_str.append(str(list_fund_avail[i]))

results = pd.DataFrame(index=list_date)
results['特殊贡献返还'] = list_特殊贡献返还
results['可用资金'] = list_fund_avail_str
results['特殊贡献返还/可用资金'] = list_ratio
results = results.drop(index=['20210625','20210928','20211227','20220325','20220627','20220927']) # exclude 特殊贡献返还-利息
results.index = pd.to_datetime(results.index)
results['year'] = results.index.year
results['month'] = results.index.month
results.index.name = '日期'
results = pd.merge(results,df_commission,left_on=['year','month'],right_index=True)
results = results.drop(columns = ['year','month'])
results

,特殊贡献返还,可用资金,特殊贡献返还/可用资金,月手续费
日期,,,,
2021-06-23,18795.53,61644801.66,0.000305,53603.71
2021-07-20,9427.99,38439243.28,0.000245,77957.23
2021-08-20,13711.50,98441654.01,0.000139,62620.51
2021-09-22,11014.73,168365345.08,0.000065,0.00
2021-11-23,3557.73,100534885.69,0.000035,62714.81
2021-12-30,8271.74,164115854.13,0.000050,14159.79
2022-01-27,4980.83,57627588.83,0.000086,6069.27
2022-06-24,68983.25,40346095.46,0.001710,20242.98
2022-07-27,4802.95,9672374.27,0.000497,15181.59


In [227]:
results.to_excel('F:\BOCIF\提取数据.xlsx')